<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/Codes/Self_supervised_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

In [ ]:
#!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [4]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 284 (delta 37), reused 35 (delta 18), pack-reused 230
Receiving objects: 100% (284/284), 74.74 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (171/171), done.


In [5]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [36]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
num_classes = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
out_dir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
memSize = 512
alpha=1

In [ ]:
#!rm -rf "ML_DL_Project"
#!rm -rf "./GTEA61"
#!rm -rf "./GTEA61_val"

In [37]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.RegObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.resnetMod import *
#from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
#Prende il makeDataset dell'ultimo script importato
import argparse
import sys

In [38]:
!rm -rf "./out_dir"

In [39]:
   model_folder = os.path.join('./', "out_dir", "./GTEA61", 'ss')
   if os.path.exists(model_folder):
        print('Directory {} exists!'.format(model_folder))
        sys.exit()
   os.makedirs(model_folder)
    
    # Log files
   writer = SummaryWriter(model_folder)
   train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
   train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
   val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
   val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

    # Data loader
   normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                                 ToTensor(), normalize])

   vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

   trainInstances = vid_seq_train.__len__()

   train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                            shuffle=True, num_workers=4, pin_memory=True)
    
   if val_data_dir is not None:
       vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')
       valInstances = vid_seq_val.__len__()

       val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize,
                                shuffle=False, num_workers=2, pin_memory=True)
    
   

In [40]:
def train(stage, learning_rate, numEpochs, decay_step, stage1_dict=None):

   train_params = []
   if stage == 1:
       model = SelfSupAttentionModel(num_classes=num_classes, mem_size=memSize)
       model.train(False)
       for params in model.parameters():

           params.requires_grad = False
   else: # stage == 2
       model = SelfSupAttentionModel(num_classes=num_classes, mem_size=memSize)
       model.load_state_dict(torch.load(stage1_dict),strict=False)
       model.train(False)
        
       for params in model.parameters():
           params.requires_grad = False
        #
       for params in model.resNet.layer4[0].conv1.parameters():
           params.requires_grad = True
           train_params += [params]

       for params in model.resNet.layer4[0].conv2.parameters():
           params.requires_grad = True
           train_params += [params]

       for params in model.resNet.layer4[1].conv1.parameters():
           params.requires_grad = True
           train_params += [params]

       for params in model.resNet.layer4[1].conv2.parameters():
           params.requires_grad = True
           train_params += [params]

       for params in model.resNet.layer4[2].conv1.parameters():
           params.requires_grad = True
           train_params += [params]
        
       for params in model.resNet.layer4[2].conv2.parameters():
           params.requires_grad = True
           train_params += [params]
        #
       for params in model.resNet.fc.parameters():
           params.requires_grad = True
           train_params += [params]

       model.resNet.layer4[0].conv1.train(True)
       model.resNet.layer4[0].conv2.train(True)
       model.resNet.layer4[1].conv1.train(True)
       model.resNet.layer4[1].conv2.train(True)
       model.resNet.layer4[2].conv1.train(True)
       model.resNet.layer4[2].conv2.train(True)
       model.resNet.fc.train(True)
    
   for params in model.lstm_cell.parameters():
       params.requires_grad = True
       train_params += [params]

   for params in model.classifier.parameters():
       params.requires_grad = True
       train_params += [params]

   model.lstm_cell.train(True)

   model.classifier.train(True)
   model.cuda()

   loss_fn = nn.CrossEntropyLoss()

   optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

   optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step,
                                                           gamma=decay_factor)

   train_iter = 0
   min_accuracy = 0
   for epoch in range(numEpochs):
       epoch_loss = 0
       numCorrTrain = 0
       mmap_loss = 0
       trainSamples = 0
       iterPerEpoch = 0
        
       model.lstm_cell.train(True)
       model.classifier.train(True)
       writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)
        
       if stage == 2:
           model.resNet.layer4[0].conv1.train(True)
           model.resNet.layer4[0].conv2.train(True)
           model.resNet.layer4[1].conv1.train(True)
           model.resNet.layer4[1].conv2.train(True)
           model.resNet.layer4[2].conv1.train(True)
           model.resNet.layer4[2].conv2.train(True)
           model.resNet.fc.train(True)
        
        #for i, (inputs, targets) in enumerate(train_loader):
       for inputs, inputMmap, targets in train_loader:
           train_iter += 1
           iterPerEpoch += 1
           optimizer_fn.zero_grad()

           inputMmap = inputMmap.cuda()
            
           inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
           labelVariable = Variable(targets.cuda())
           trainSamples += inputs.size(0)
            
           output_label, _ , mmapPrediction = model(inputVariable)

           mmapPrediction = mmapPrediction.view(-1,2)         
           inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
           inputMmap = torch.round(inputMmap).long() #making things black and white again
            

           loss2 = alpha*loss_fn(mmapPrediction,inputMmap)
           loss = loss_fn(output_label, labelVariable)

            #Weighting the loss of the ss task
            #by multiplying it by alpha
           total_loss = loss  + loss2
           total_loss.backward()
            
           optimizer_fn.step()
           _, predicted = torch.max(output_label.data, 1)
           numCorrTrain += (predicted == targets.cuda()).sum()
           mmap_loss += loss2.item()
           epoch_loss += loss.item()
        
       optim_scheduler.step()
       avg_mmap_loss = mmap_loss / iterPerEpoch
       avg_loss = epoch_loss/iterPerEpoch
       trainAccuracy = torch.true_divide(numCorrTrain , trainSamples) * 100

       print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))
       print('Mmap loss after {} epoch = {}% '.format(epoch + 1, avg_mmap_loss))

       writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
       writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)
       writer.add_scalar('mmap_train_loss',avg_mmap_loss,epoch+1)

       train_log_loss.write('Training mmap loss after {} epoch= {}'.format(epoch+1,avg_mmap_loss))
       train_log_loss.write('Training loss after {} epoch = {}\n'.format(epoch+1, avg_loss))
       train_log_acc.write('Training accuracy after {} epoch = {}\n'.format(epoch+1, trainAccuracy))
        
       if valDatasetDir is not None:
           model.train(False)
           val_loss_epoch = 0
           val_iter = 0
           val_mmap_loss = 0
           val_samples = 0
           numCorr = 0
           mmap_loss = 0

           with torch.no_grad():
                #for j, (inputs, targets) in enumerate(val_loader):
               for inputs, inputMmap, targets in val_loader:
                   val_iter += 1
                   val_samples += inputs.size(0)
                   inputMmap = inputMmap.cuda()

                    
                   inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
                   labelVariable = Variable(targets.cuda(async=True))
                    #labelVariable = Variable(targets.cuda())
                    
                   output_label, _ , mmapPrediction = model(inputVariable)

                   mmapPrediction = mmapPrediction.view(-1,2)
                   inputMmap = torch.reshape(inputMmap, (-1,)) #.long()
                   inputMmap = torch.round(inputMmap).long()
                   loss2 = alpha*loss_fn(mmapPrediction,inputMmap)

                   val_loss = loss_fn(output_label, labelVariable)
                   val_loss_epoch += val_loss.item()
                   val_mmap_loss += loss2.item()
                    
                   _, predicted = torch.max(output_label.data, 1)
                   numCorr += (predicted == targets.cuda()).sum()

           avg_mmap_val_loss = val_mmap_loss / val_iter
           val_accuracy = torch.true_divide(numCorr , val_samples) * 100
           avg_val_loss = val_loss_epoch / val_iter
            
           print('Val MMap Loss after {} epochs, loss = {}'.format(epoch + 1, avg_mmap_val_loss))
           print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
           writer.add_scalar('val mmap/epoch_loss', avg_mmap_val_loss, epoch + 1)
           writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
           writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
           val_log_loss.write('Val MMap Loss after {} epochs = {}\n'.format(epoch + 1, avg_mmap_val_loss))
           val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))
           val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))
            
           if val_accuracy > min_accuracy:
               save_path_model = (model_folder + '/model_rgb_state_dict.pth')
               torch.save(model.state_dict(), save_path_model)
               min_accuracy = val_accuracy

   train_log_loss.close()
   train_log_acc.close()
   val_log_acc.close()
   val_log_loss.close()
   writer.export_scalars_to_json(model_folder + "/all_scalars.json")
   writer.close()

In [41]:
train(1, lr1, numEpochs1, decay_step1)

train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
#writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.flush()
writer.close()

ValueError: ignored

In [ ]:
writer = SummaryWriter('runs/exp1/stage/2')
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')


train(2, lr2, numEpochs2, decay_step2, stage1_dict="./out_dir/GTEA61/rgb/model_rgb_state_dict.pth")

train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
#writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.flush()
writer.close()

%tensorboard --logdir=runs/exp1
#!rm -rf ./runs/exp1/stage
#Using it only if run again, in view to clean the folders